In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
import keras.applications
import os
from keras.layers import merge, Input
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


In [2]:
TRAIN_FOLDER  = 'train/'
TEST_FOLDER  = 'test/'
train_df = pd.read_csv('labels.csv')

top_breeds = sorted(list(train_df['breed'].value_counts().head(20).index))
# top_breeds = sorted(list(train_df['breed']))
train_df = train_df[train_df['breed'].isin(top_breeds)]

In [3]:
batch_size = 64
SEED = 9
transformation_ratio = .05

In [4]:
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split

DIM = 224

train_df['image_path'] = train_df.apply( lambda x: ( TRAIN_FOLDER + x["id"] + ".jpg" ), axis=1 )

train_data = np.array([ img_to_array(load_img(img, target_size=(DIM, DIM))) for img in train_df['image_path'].values.tolist()]).astype('float32')
train_labels = train_df['breed']


x_train, x_validation, y_train, y_validation = train_test_split(train_data, train_labels, test_size=0.2, stratify=np.array(train_labels), random_state=SEED)


In [5]:
y_train = pd.get_dummies(y_train.reset_index(drop=True), columns=top_breeds).as_matrix()
y_validation = pd.get_dummies(y_validation.reset_index(drop=True), columns=top_breeds).as_matrix()

In [6]:
breed_nums = len(y_train[0])

In [7]:
train_datagen = ImageDataGenerator(rescale=1., 
                                   rotation_range=30, 
                                   # zoom_range = 0.3, 
                                   width_shift_range=0.2,
                                   height_shift_range=0.2, 
                                   horizontal_flip = 'true')
train_generator = train_datagen.flow(x_train, y_train, shuffle=False, batch_size=batch_size, seed=10)

val_datagen = ImageDataGenerator(rescale = 1.)
val_generator = train_datagen.flow(x_validation, y_validation, shuffle=False, batch_size=batch_size, seed=10)

In [8]:
train_generator.n

1747

In [8]:
import time

image_input = Input(shape=(224, 224, 3))
base_model = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
x = base_model.output
# x = Flatten()(x)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dropout(0.25, name='Dropout1')(x)
x = Dense(1024, activation='relu', name='fc2')(x)
# x = Dropout(0.5, name='Dropout2')(x)
predictions = Dense(breed_nums, activation='softmax', name='output')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
# optimizer1 = Adam(lr=0.0001, decay=0.005)
# optimizer2 = Adam(lr=0.00005, decay=0.005)

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

histroy = model.fit_generator(train_generator,
                              steps_per_epoch = train_generator.n/batch_size,
                              validation_data = val_generator,
                              validation_steps = val_generator.n/batch_size,
                              epochs = 5,
                              verbose = 2)
model.save('VGG16_transferL.h5')

Epoch 1/5
 - 21s - loss: 4.7737 - acc: 0.3109 - val_loss: 2.5835 - val_acc: 0.5034
Epoch 2/5
 - 14s - loss: 2.0085 - acc: 0.5778 - val_loss: 1.8658 - val_acc: 0.5789
Epoch 3/5
 - 15s - loss: 1.6141 - acc: 0.6831 - val_loss: 1.6744 - val_acc: 0.6293
Epoch 4/5
 - 15s - loss: 0.9909 - acc: 0.7234 - val_loss: 1.1006 - val_acc: 0.6384
Epoch 5/5
 - 15s - loss: 0.7251 - acc: 0.7580 - val_loss: 1.0459 - val_acc: 0.6842


In [12]:
for layer in model.layers[:6]:
   layer.trainable = False
for layer in model.layers[6:]:
   layer.trainable = True

optimizer2 = Adam(lr=0.00005, decay=0.005)
epoch_num = 10
earlystopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=2, verbose=0, mode='auto')

#Begin Model Traininga
model.compile(loss='categorical_crossentropy',optimizer=optimizer2,metrics=['accuracy'])
t=time.time()
#	t = now()
train_history = model.fit_generator(train_generator,
                              steps_per_epoch = train_generator.n/batch_size,
                              validation_data = val_generator,
                              validation_steps = val_generator.n/batch_size,
                              epochs = 5,
                              verbose = 2)
print('Training time: %s mins' % round((time.time()-t)/60, 2))

model.save('VGG16_Finetuning3_avg_do0.25.h5')

Epoch 1/5
 - 23s - loss: 0.7975 - acc: 0.7428 - val_loss: 0.8390 - val_acc: 0.7414
Epoch 2/5
 - 21s - loss: 0.4541 - acc: 0.8391 - val_loss: 0.7361 - val_acc: 0.7574
Epoch 3/5
 - 21s - loss: 0.3533 - acc: 0.8803 - val_loss: 0.7940 - val_acc: 0.7826
Epoch 4/5
 - 21s - loss: 0.3265 - acc: 0.8947 - val_loss: 0.6695 - val_acc: 0.8009
Epoch 5/5
 - 21s - loss: 0.2045 - acc: 0.9347 - val_loss: 0.9723 - val_acc: 0.7712
Training time: 1.79 mins


In [13]:
preds = model.predict(x_test, verbose=1)

df_test = pd.read_csv('sample_submission.csv')

sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', df_test['id'])
sub.head(5)

NameError: name 'x_test' is not defined